In [23]:
import numpy as np
import tensorflow as tf
import keras

# Structure for detection task

1. Dataloaders:
    - Input images for predicton of detection task: 
         * preprocess_input(): process original images to image data
         * decoder(): convert image data to tensor
         * parser(): breaks the input into parts
    - Load pre-trained weights into YOLOv3 model:
        * (utils) download_weights()
        * load_weights_to_model()
 2. Modeling:
     - nn_block() or layers:
     - yolo_model():
 3. Ops:
     - decode_netout(): decode the prediction of detection model into boxes 
     - correct_boxes():
     - nms() and bbox_iou():
     - draw_boxes():
 4. Configs:
     - Image Input:
         * image_path:
         * num_classes:
         * input_size: for modeling
     - Pretrained weights:
         * pb_file: pretrain weights
 
 5. Tasks:
     - build_input():
     - build_model():
     - build_loss():
     - metrics:
 
 6. Common/Import Registry:
     - Include the components above
 
 7. Train:
     - train.py
 
 8. Demo

In [3]:
!pip install tf-nightly

INFO: pip is looking at multiple versions of tf-nightly to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/212.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.4 MB 1.9 MB/s eta 0:01:51
   ---------------------------------------- 0.0/212.4 MB 1.9 MB/s eta 0:01:51
   ---------------------------------------- 0.1/212.4 MB 1.1 MB/s eta 0:03:20
   ---------------------------------------- 0.1/212.4 MB 1.1 MB/s eta 0:03:20
   ---------------------------------------- 0.4/212.4 MB 1.7 MB/s eta 0:02:04
   ---------------------------------------- 0.8/212.4 MB 2.8 MB/s eta 0:01:16
   ---------------------------------------- 1.0/212.4 MB 3.6 MB/s eta 0:00:59
   ---------------------------------------- 1.2/212.4 MB 3.6 MB/s eta 0:00:59
   ---------------------------------------- 1.6/212.4 MB 4.1 MB/s eta 0:00:52
   ---------------------------------------- 1.6/212.4 MB 4.2 MB/s eta 0:00:51
  

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\acer\\anaconda3\\envs\\Monument\\Lib\\site-packages\\tensorflow\\compiler\\mlir\\quantization\\tensorflow\\calibrator\\pywrap_calibration.pyd'
Consider using the `--user` option or check the permissions.



   --------------- ------------------------ 80.0/212.4 MB 1.2 MB/s eta 0:01:54
   --------------- ------------------------ 80.1/212.4 MB 1.2 MB/s eta 0:01:54
   --------------- ------------------------ 80.1/212.4 MB 1.2 MB/s eta 0:01:53
   --------------- ------------------------ 80.2/212.4 MB 1.2 MB/s eta 0:01:51
   --------------- ------------------------ 80.3/212.4 MB 1.2 MB/s eta 0:01:51
   --------------- ------------------------ 80.3/212.4 MB 1.2 MB/s eta 0:01:51
   --------------- ------------------------ 80.4/212.4 MB 1.2 MB/s eta 0:01:50
   --------------- ------------------------ 80.4/212.4 MB 1.2 MB/s eta 0:01:50
   --------------- ------------------------ 80.5/212.4 MB 1.2 MB/s eta 0:01:50
   --------------- ------------------------ 80.5/212.4 MB 1.2 MB/s eta 0:01:50
   --------------- ------------------------ 80.7/212.4 MB 1.2 MB/s eta 0:01:49
   --------------- ------------------------ 80.8/212.4 MB 1.2 MB/s eta 0:01:49
   --------------- ------------------------ 80.9/212

##### Model Garden Packages

In [4]:
!pip install tf-models-official

     ---------------------------------------- 0.0/84.6 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/84.6 kB ? eta -:--:--
     ------------- ------------------------ 30.7/84.6 kB 435.7 kB/s eta 0:00:01
     --------------------------- ---------- 61.4/84.6 kB 544.7 kB/s eta 0:00:01
     -------------------------------------- 84.6/84.6 kB 675.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pycocotools-2.0.7-cp312-cp312-win_amd64.whl
  Using cached sacrebleu-2.4.1-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB 1.0 MB/s eta 0:00:00
  Preparing me

  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [54 lines of output]
  running egg_info
  writing lib3\PyYAML.egg-info\PKG-INFO
  writing dependency_links to lib3\PyYAML.egg-info\dependency_links.txt
  writing top-level names to lib3\PyYAML.egg-info\top_level.txt
  Traceback (most recent call last):
    File "C:\Users\acer\anaconda3\envs\Monument\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\acer\anaconda3\envs\Monument\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\acer\anaconda3\envs\Monument\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
      return hook(config_settings)
   

In [5]:
!pip install tf-models-nightly

  Using cached Cython-3.0.10-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
  Using cached pillow-10.3.0-cp312-cp312-win_amd64.whl.metadata (9.4 kB)
  Using cached gin_config-0.5.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached google_api_python_client-2.125.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached immutabledict-4.2.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached kaggle-1.6.8.tar.gz (84 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached matplotlib-3.8.4-cp312-cp312-win_amd64.whl.metadata (5.9 kB)
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pycocotools-2.0.7-cp312-cp312-win_amd64.whl
  Using cached PyYAML-6.0.1-cp312-cp312-win_amd6

  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [54 lines of output]
  running egg_info
  writing lib3\PyYAML.egg-info\PKG-INFO
  writing dependency_links to lib3\PyYAML.egg-info\dependency_links.txt
  writing top-level names to lib3\PyYAML.egg-info\top_level.txt
  Traceback (most recent call last):
    File "C:\Users\acer\anaconda3\envs\Monument\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\acer\anaconda3\envs\Monument\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\acer\anaconda3\envs\Monument\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
      return hook(config_settings)
   

In [6]:
!pip install tensorflow-datasets

  Using cached tensorflow_datasets-4.9.4-py3-none-any.whl.metadata (9.2 kB)
  Using cached dm_tree-0.1.8-cp312-cp312-win_amd64.whl.metadata (2.0 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     -------------------- ----------------- 30.7/57.6 kB 435.7 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/57.6 kB 435.7 kB/s eta 0:00:01
     --------------------------- ---------- 41.0/57.6 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 251.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
    --------------------------------------- 0.1/5.1 MB ? eta -:--:--
    --------------------------------------- 0.1/5.1 MB ? eta -:--:--
    --------------------------------------- 0.1/5.1 MB ? eta -:--:--
    ----

  You can safely remove it manually.


In [7]:
!pip install tfds-nightly

  Using cached immutabledict-4.2.0-py3-none-any.whl.metadata (3.4 kB)
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB 991.0 kB/s eta 0:00:06
    --------------------------------------- 0.1/5.1 MB 1.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.3/5.1 MB 2.4 MB/s eta 0:00:03
   --- ------------------------------------ 0.4/5.1 MB 2.5 MB/s eta 0:00:02
   ----- ---------------------------------- 0.7/5.1 MB 3.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.7/5.1 MB 3.3 MB/s eta 0:00:02
   -------- ------------------------------- 1.1/5.1 MB 3.6 MB/s eta 0:00:02
   ---------- ----------------------------- 1.3/5.1 MB 3.9 MB/s eta 0:00:01
   ------------ --------------------------- 1.5/5.1 MB 3.9 MB/s eta 0:00:01
   --------------- ------------------------ 1.9/5.1 MB 4.3 MB/s eta 0:00:01
   ------------------ --------

In [8]:
!pip install blosc2~=2.0.0

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB 262.6 kB/s eta 0:00:12
     ---------------------------------------- 0.0/3.0 MB 262.6 kB/s eta 0:00:12
      --------------------------------------- 0.1/3.0 MB 393.8 kB/s eta 0:00:08
     - -------------------------------------- 0.1/3.0 MB 554.9 kB/s eta 0:00:06
     --- ------------------------------------ 0.2/3.0 MB 901.1 kB/s eta 0:00:04
     ---- ----------------------------------- 0.4/3.0 MB 1.2 MB/s eta 0:00:03
     ------- -------------------------------- 0.6/3.0 MB 1.7 MB/s eta 0:00:02
     ---------- ----------------------------- 0.8/3.0 MB 2.1 MB/s eta 0:00:02
     ---------- ----------------------------- 0.8/3.0 MB 2.1 MB/s eta 0:00:02
     ------------ --------------------------- 0.9/3.0 MB 2.0 MB/s eta 0:00:02
     -------------- ------------------------- 1.1/3.0 MB 2.2 MB/s et

  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 2
  
  [1324 lines of output]
    Using cached setuptools-69.2.0-py3-none-any.whl.metadata (6.3 kB)
    Using cached Cython-3.0.10-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
    Using cached oldest_supported_numpy-2023.12.21-py3-none-any.whl.metadata (9.8 kB)
    Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
    Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
    Using cached numpy-1.26.2-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached setuptools-69.2.0-py3-none-any.whl (821 kB)
     ---------------------------------------- 0.0/84.3 kB ? eta -:--:--
     -------------- ------------------------- 30.7/84.3 kB 640.0 kB/s eta 0:00:01
     -------------- ------------------------- 30.7/84.3 kB 640.0 kB/s eta 0:00:01
     ------------------- -------------------- 41.0/84.3 kB 245.8 kB/s eta 0:00:01
     -------------------------

In [9]:
!pip install --upgrade tables

  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ------------------------------

In [11]:
!pip install cython

  Using cached Cython-3.0.10-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
Using cached Cython-3.0.10-cp312-cp312-win_amd64.whl (2.8 MB)


In [12]:
!pip install requests_mock

  Using cached requests_mock-1.12.1-py2.py3-none-any.whl.metadata (4.1 kB)
Using cached requests_mock-1.12.1-py2.py3-none-any.whl (27 kB)


In [18]:
!C:\Users\acer\anaconda3\python.exe -m pip install --upgrade pip setuptools

## Data Loader

Reads, Decodes and Parses the input data. Contains a decoder and Parser.

Decoder decodes a TF record and returns a dictionary of decoded tensors. 

In [26]:
import orbit
import tensorflow_models as tfm

from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

pp = pprint.PrettyPrinter(indent=4) # Set Pretty Print Indentation
print(tf.__version__) # Check the version of tensorflow used

%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow_models'

In [20]:
class YOLODecoder(decoder.Decoder): # inherits from decoder.Decoder provided by Tensorflow Model Garden
    def __init__(self):
        """Initializes the decoder.
        Defines mapping between the field name and value from an input. 
        E.g. We define two fields for image bytes and labels."""
        
        # defines a dictionary that contains mapping between feature keys and 
        # their corresponding feature types.
        self._keys_to_features = {
            # FixedLenFeature is used when the feature has a fixed length. It expexts
            # a single value or a tensor of a specified shape and data type.
            # 'image/encoded' is a fixed-length feature representing the encoded image data. 
            # The shape is specified as '()' since its a single string (the image data), and the data type is 
            # 'tf.string'.
            # VarLenFeature is used when the feature has a variable length. It expects a
            # sparse tensor representing a list of values of the specified data type. 
            # 'image/object/bbox/xmin', 'image/object/bbox/ymin', 'image/object/bbox/xmax', 
            # 'image/object/bbox/ymax', and 'image/object/class/label' are variable-length features.
            'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
            #'image/class/label': tf.io.FixedLenFeature((), tf.int64, default_value = -1)
            # Since the number of bounding boxes and labels can vary from image to image, 
            # it's more efficient to use variable-length features. 
            'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
            'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
            'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
            'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
            'image/object/class/label': tf.io.VarLenFeature(tf.float32),
        }
        
    def decode(self, features): # Features in a dictionary containing decoded tensors
        
        # decode image
        
        # decodes the encoded image data ('image/encoded') using tf.image.decode_jpeg. decodes jpeg-encoded images into uint8 tensors. channels = 3 indicates RGB.
        image = tf.image.decode_jpeg(features['image/encoded'], channels = 3)
        
        # Decode the bounding boxes
        
        # decodes sparse tensors to dense tensors
        xmin = tf.sparse.to_dense(features['image/object/bbox/xmin'])
        ymin = tf.sparse.to_dense(features['image/object/bbox/ymin'])        
        xmax = tf.sparse.to_dense(features['image/object/bbox/xmax'])        
        ymax = tf.sparse.to_dense(features['image/object/bbox/ymax'])        
        labels = tf.sparse.to_dense(features['image/object/class/label'])        
        
        
        # Combine bounding box coordinates
        
        # stacks the decoded bounding box coordinates ('xmin', 'ymin', 'xmax', 'ymax') along the last axis
        # ('axis = -1') to form a tensor of shape '(num_boxes, 4)'.
        # Each row of this tensor represents a bounding box with coordinates '[xmin, ymin, xmax, ymax]'
        boxes = tf.stack([xmin, ymin, xmax, ymax], axis = -1)
        
        return image, boxes, labels

NameError: name 'decoder' is not defined